In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import copy
import yaml
import os
from os.path import dirname
import sys
import argparse
import numpy as np
import math
import numpy as np

root=os.getcwd()
print(root)
sys.path.append(root)
sys.path.append("..")

from STAEformer import STAEformer
from lib.data_prepare import get_dataloaders_from_index_data
from train import RMSE_MAE_MAPE

/Users/wei/Desktop/Zhangmingchao_Code/model


In [6]:
reuse_model_path=f"../saved_models/STAEformer-PEMS08-2025-10-27-21-58-25.pt"
checkpoint = torch.load(reuse_model_path, map_location="cpu")
for k,v in checkpoint.items():
    print(f"layer: {k}, parameters: {v.shape}")

layer: adaptive_embedding, parameters: torch.Size([12, 170, 80])
layer: input_proj.weight, parameters: torch.Size([24, 3])
layer: input_proj.bias, parameters: torch.Size([24])
layer: tod_embedding.weight, parameters: torch.Size([288, 24])
layer: dow_embedding.weight, parameters: torch.Size([7, 24])
layer: output_proj.weight, parameters: torch.Size([12, 1824])
layer: output_proj.bias, parameters: torch.Size([12])
layer: attn_layers_t.0.attn.FC_Q.weight, parameters: torch.Size([152, 152])
layer: attn_layers_t.0.attn.FC_Q.bias, parameters: torch.Size([152])
layer: attn_layers_t.0.attn.FC_K.weight, parameters: torch.Size([152, 152])
layer: attn_layers_t.0.attn.FC_K.bias, parameters: torch.Size([152])
layer: attn_layers_t.0.attn.FC_V.weight, parameters: torch.Size([152, 152])
layer: attn_layers_t.0.attn.FC_V.bias, parameters: torch.Size([152])
layer: attn_layers_t.0.attn.out_proj.weight, parameters: torch.Size([152, 152])
layer: attn_layers_t.0.attn.out_proj.bias, parameters: torch.Size([15

In [11]:
in_steps=12
model_dim=152
out_steps=12
output_dim=1
predictor=nn.Linear(in_steps*model_dim, out_steps*output_dim)
predictor_dict=predictor.state_dict()
reuse_layers = ["output_proj.weight", "output_proj.bias"]

pretrained_dict = {k: v for k, v in checkpoint.items()
                   if any(layer in k for layer in reuse_layers)}
predictor_dict.update(pretrained_dict)
predictor.load_state_dict(predictor_dict, strict=True)

data=torch.randn((16,170,1824))
print(predictor(data).shape)



RuntimeError: Error(s) in loading state_dict for Linear:
	Unexpected key(s) in state_dict: "output_proj.weight", "output_proj.bias". 